In [1]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import load_dataset
import torch
import os
from jiwer import wer, cer 

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
print(torch.cuda.is_available())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

True


In [3]:
# загрузка модели и токенизатора
processor = Wav2Vec2Processor.from_pretrained("bond005/wav2vec2-large-ru-golos")
model = Wav2Vec2ForCTC.from_pretrained("doooori/wav2vec2_pisets")

Downloading: 100%|██████████| 1.26G/1.26G [1:06:05<00:00, 318kB/s] 


In [4]:
# загрузка датасета
ds = load_dataset("MegaKosT/RuDevSberDS", split="test")

Using custom data configuration MegaKosT--RuDevSberDS-c815bd8b35ec831d
Found cached dataset parquet (/root/.cache/huggingface/datasets/MegaKosT___parquet/MegaKosT--RuDevSberDS-c815bd8b35ec831d/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [5]:
ds = ds.filter(
    lambda it1: (it1["transcription"] is not None) and (len(it1["transcription"].strip()) > 0)
)

Loading cached processed dataset at /root/.cache/huggingface/datasets/MegaKosT___parquet/MegaKosT--RuDevSberDS-c815bd8b35ec831d/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-f925d8d7dc0d094e.arrow


In [7]:
import random
# токенизируем
rand_int = random.randint(0, len(ds)-1)
processed = processor(ds[rand_int]["audio"]["array"], return_tensors="pt", padding="longest")

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [8]:
# извлекаем логиты
logits = model(processed.input_values, attention_mask=processed.attention_mask).logits

In [9]:
# берем argmax и декодируем
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)[0]
print(transcription)

джой посчитай мне двадцать пять плюс семнадцать сколько будет


In [10]:
ds[rand_int]

{'audio': {'path': '783be89657e6107d78935a5aba832bac.wav',
  'array': array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         3.05175781e-05, 0.00000000e+00, 0.00000000e+00]),
  'sampling_rate': 16000},
 'transcription': 'джой посчитай мне двадцать пять плюс семнадцать сколько будет'}

In [11]:
model.to(device)

# recognize one sound
def map_to_pred(batch):
    # tokenize and vectorize
    processed = processor(
        batch["audio"]["array"], sampling_rate=batch["audio"]["sampling_rate"],
        return_tensors="pt", padding="longest"
    )
    input_values = processed.input_values.to(device)
    attention_mask = processed.attention_mask.to(device)

    # recognize
    with torch.no_grad():
        logits = model(input_values, attention_mask=attention_mask).logits
    predicted_ids = torch.argmax(logits, dim=-1)

    # decode
    transcription = processor.batch_decode(predicted_ids)
    batch["text"] = transcription[0]
    return batch

# calculate WER and CER
result = ds.map(map_to_pred, remove_columns=["audio"])
wer = wer(result["transcription"], result["text"])
cer = cer(result["transcription"], result["text"])
print("Word error rate:", wer)
print("Character error:", cer)

100%|██████████| 9194/9194 [03:52<00:00, 39.49ex/s]


Word error rate: 0.1777212885813366
Character error: 0.05889145808824524
